In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from tqdm import tqdm
from embedding_model import Embedder
import chromadb
import torch





In [2]:
def augment_multiple_query(query, model="gpt-3.5-turbo"):
    messages = [
        {
            "role": "system",
            "content": SYSTEM_PROMT_QUERY_EXPANSION,
        },
        {"role": "user", "content": query}
    ]

    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    content = content.split("\n")
    return content

### Usage


```bash
docker run tinyrag --persistent_storage [path]  
```


```bash
docker-compose up
```
### This should deploys a RAG API with 4 endpoints:

 - /tinyrag/upload_file
 
 - /tinyrag/upload_zip
 
 - /tinyrag/query$?expand
 
 - /tinyrag/reset
 
 
 Think more about it

In [15]:
class PdfChunksLoader_ChromaDB():
    def __init__(self, collection, embedder, text_splitter=None):
        
        
        
        self.collection = collection
        self.embedder = embedder
        self.id = 0
        self.text_splitter = text_splitter if text_splitter else RecursiveCharacterTextSplitter(chunk_size=1500, 
                                                                           chunk_overlap=100,
                                                                           separators=["\n", "\t", ".", ",", " ", ""],)
        
        
    def _extract_pdf_chunks(self, path):
        
        loader = PyPDFLoader(path)
        
        chunks = loader.load_and_split(text_splitter=self.text_splitter)
        
        return chunks
        
        
    def populate(self, documents):
        ##TODO: add batch size for computing embeddings
        
        self.collection.add(
            documents=[chunk.page_content for chunk in tqdm(documents)],
            
            metadatas = [chunk.metadata for chunk in tqdm(documents)],
            
            embeddings = self.embedder.compute_embeddings([chunk.page_content for chunk in documents]).tolist(),
            ids = ["uri" + str(abs(hash(chunk.page_content))) for chunk in tqdm(documents)]
            
            )
 
        print("Documents loaded to DB")

In [20]:


#Works for single collection
class RetrievalAugmentedGenerator():
    def __init__(self, db_client, embedder, collection_name):
        
        self.db_client = db_client
        self.embedder = embedder
        
        
        
        self.collection_name = collection_name
        self.collection = self.db_client.get_or_create_collection(name=self.collection_name)
        
        self.chunk_loader = PdfChunksLoader_ChromaDB(self.collection,
                                                     embedder)

    def upload_pdf_file(self, path_file):
        ##Load chunks by batches
        
        docs = self.chunk_loader._extract_pdf_chunks(path_file)
        
        self.chunk_loader.populate(documents=docs)
        
    
    def upload_documents_from_list(self, documents):
        pass
    
    def upload_from_dir(self, path_dir):
        ##Load chunks by batches
        pass
    
    
    
    
    def query(self, queries, top_k, query_augment=True, cross_encoder_rank=True):
        pass
    
    def get(self, ids, where, limit):
        pass
    
    def reset_collection(self):
        pass
    
    
    
    

In [17]:
db_client = chromadb.PersistentClient(path="./persistent_storage")

#collection = db_client.get_collection("my_collection")
embedder =  Embedder(model_name='sentence-transformers/all-MiniLM-L12-v2',
                    tokenizer_name='sentence-transformers/all-MiniLM-L12-v2')



In [21]:
rag = RetrievalAugmentedGenerator(db_client, embedder, "hello")

In [ ]:
rag.upload_pdf_file("library/pthreads.pdf")

100%|███████████████████████████████████████| 66/66 [00:00<00:00, 236601.76it/s]
